In [1]:
# import face_recognition
import unittest
import os
import numpy as np
from click.testing import CliRunner

from face_recognition import api
from face_recognition import face_recognition_cli
from face_recognition import face_detection_cli

import os
import numpy as np
from face_recognition import api
from sklearn.cluster import DBSCAN
from collections import defaultdict

In [2]:
def get_face_encodings(folder_path):
    encodings = []
    file_paths = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        image = api.load_image_file(file_path)
        face_encodings = api.face_encodings(image)
        if face_encodings:
            encodings.append(face_encodings[0])  # Assuming one face per image
            file_paths.append(file_path)
    return encodings, file_paths

known_folder = "photos/known_faces"
unknown_folder = "photos/unknown_faces"

known_encodings, known_paths = get_face_encodings(known_folder)
unknown_encodings, unknown_paths = get_face_encodings(unknown_folder)

all_encodings = known_encodings + unknown_encodings
print(len(all_encodings[0]))


128


In [5]:
print(all_encodings[0])

[-5.14035411e-02  7.78007656e-02 -7.85855949e-03  7.83444010e-03
 -1.04334146e-01  1.88750494e-02  5.40590659e-03 -1.00495167e-01
  1.63838074e-01 -1.55132219e-01  3.01148564e-01 -4.08388637e-02
 -3.37635368e-01 -4.95465659e-02 -6.79349005e-02  1.23000219e-01
 -7.39021301e-02 -1.06222264e-01 -7.18452409e-02 -1.54797629e-01
  1.17018715e-01  7.72570968e-02 -1.05661247e-02  2.78139953e-02
 -1.90182030e-01 -2.26765171e-01 -8.17275122e-02 -1.17221847e-01
  7.23719671e-02 -1.48138121e-01  1.68969519e-02  8.68462622e-02
 -1.09084748e-01 -1.73151512e-02  6.84938803e-02  9.81915146e-02
 -5.40035293e-02 -4.60427143e-02  2.62658477e-01  8.48521665e-03
 -1.87697917e-01 -5.07488549e-02  3.03445645e-02  2.81404823e-01
  1.33905843e-01  1.86462011e-02  5.42550534e-03 -8.28128457e-02
  1.49981990e-01 -1.97875351e-01  7.08558708e-02  1.29258633e-01
  2.35844627e-01  8.58479589e-02  8.45161527e-02 -2.78427392e-01
  8.74418765e-05  1.69854417e-01 -2.39883780e-01  2.76353918e-02
 -4.95622866e-03 -9.41139

In [3]:
db = DBSCAN(eps=0.5, min_samples=2, metric="euclidean").fit(all_encodings)
labels = db.labels_

clusters = defaultdict(list)
for idx, label in enumerate(labels):
    image_path = known_paths[idx] if idx < len(known_encodings) else unknown_paths[idx - len(known_encodings)]
    clusters[label].append(image_path)

for label, images in clusters.items():
    if label == -1:
        print("Unclustered faces:")
    else:
        print(f"Cluster {label}:")
    for image_path in images:
        print(f"  - {image_path}")


Cluster 0:
  - photos/known_faces\IMG_5401.jpg
  - photos/unknown_faces\image0.jpg
  - photos/unknown_faces\image1.jpg
  - photos/unknown_faces\image2.jpg
Cluster 1:
  - photos/known_faces\rn_image_picker_lib_temp_820ef847-389d-4b67-9a7a-f36edc967bbc.jpg
  - photos/unknown_faces\20240413_122016.jpg
  - photos/unknown_faces\20241006_162622.jpg
Cluster 2:
  - photos/known_faces\Screenshot 2024-10-24 221400.png
  - photos/unknown_faces\IMG_2747.jpg
Unclustered faces:
  - photos/known_faces\Screenshot 2024-10-24 221507.png
  - photos/unknown_faces\20240324_023250.jpg
  - photos/unknown_faces\IMG_2488.jpg
  - photos/unknown_faces\IMG_2617.png
  - photos/unknown_faces\IMG_3721.jpg
